In [15]:
#env: equake_env (Python 3.9)
import pandas as pd
import sqlite3
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [16]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


In [17]:
#create and connect to a sqlite3 db
path = "../Database/db/earthquake_db"
conn = sqlite3.connect(path)

Adding 1 month of data

In [18]:
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-06-27T22:27:26.030Z,33.230333,-116.726000,11.43,0.86,ml,33.0,54.0,0.02803,0.15,...,2022-06-27T22:31:01.524Z,"4km ESE of Lake Henshaw, CA",earthquake,0.26,0.61,0.156,22.0,automatic,ci,ci
1,2022-06-27T22:25:42.650Z,39.434500,-110.322333,-0.85,1.40,md,7.0,176.0,0.01061,0.04,...,2022-06-27T22:38:14.130Z,"14 km SSE of Sunnyside, Utah",earthquake,0.90,0.25,0.193,6.0,reviewed,uu,uu
2,2022-06-27T22:24:18.380Z,33.232000,-116.728167,11.10,0.54,ml,16.0,41.0,0.02595,0.17,...,2022-06-27T22:27:45.505Z,"3km ESE of Lake Henshaw, CA",earthquake,0.45,0.73,0.208,11.0,automatic,ci,ci
3,2022-06-27T22:21:10.484Z,59.060800,-152.825400,69.40,1.40,ml,NaN,NaN,NaN,0.60,...,2022-06-27T22:23:46.125Z,"61 km WSW of Nanwalek, Alaska",earthquake,NaN,0.70,NaN,NaN,automatic,ak,ak
4,2022-06-27T22:19:48.820Z,38.816334,-122.762169,1.92,0.66,md,12.0,113.0,0.01479,0.02,...,2022-06-27T22:28:10.958Z,"3km W of Cobb, CA",earthquake,0.42,0.74,0.150,2.0,automatic,nc,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8993,2022-05-28T22:58:29.229Z,61.038300,-148.479600,24.20,1.40,ml,NaN,NaN,NaN,0.82,...,2022-05-28T23:11:56.523Z,"31 km NNE of Whittier, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
8994,2022-05-28T22:44:31.860Z,37.602500,-118.818500,3.47,1.22,md,16.0,94.0,0.01503,0.03,...,2022-05-29T22:09:10.832Z,"13km WNW of Toms Place, CA",earthquake,0.37,0.31,0.097,14.0,reviewed,nc,nc
8995,2022-05-28T22:41:00.294Z,56.730700,-156.474800,0.00,2.10,ml,NaN,NaN,NaN,1.11,...,2022-06-12T03:11:38.040Z,"103 km SSE of Ugashik, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
8996,2022-05-28T22:40:51.511Z,60.886100,-151.031200,24.60,1.00,ml,NaN,NaN,NaN,0.54,...,2022-05-28T22:52:29.667Z,"19 km WSW of Point Possession, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak


In [19]:
#creates/replaces database to add new data to json file
#=============
#PostgreSQL
df.to_sql('earthquake_update', engine, if_exists='replace', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_update', conn, if_exists='replace', index=False)

8998

Load from database to update with new rows and remove duplicate rows

In [23]:
#Dataframe from postgresql
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df.shape

(64462, 22)

In [24]:
#dataframe from SQLite
model_df2 = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df2.shape

(64207, 22)

In [30]:
#concatenate new data df with data read from postgres db table
model_df = pd.concat([model_df,df], verify_integrity=True, ignore_index=True)
model_df.shape

(73460, 22)

In [32]:
#concatenate new data df with data read from sqlite db table
model_df2 = pd.concat([model_df2,df], verify_integrity=True, ignore_index=True)
model_df2.shape

(73205, 22)

In [31]:
#check for duplicates in postgres data
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

True


In [33]:
#check for duplicates in sqlite data
boolean = model_df2.duplicated(subset=['id']).any()
print(boolean)

True


In [34]:
#drop duplicates in postgres data
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time  latitude  longitude   depth  mag magType  \
0      2022-01-06T01:42:39.855Z   37.3840  -117.1075    1.10  1.3      ml   
1      2022-01-06T01:31:33.698Z   38.1810  -117.8180    7.30  1.4      ml   
2      2022-01-06T01:24:20.083Z   61.3407  -149.8852   30.00  1.5      ml   
3      2022-01-06T01:15:15.851Z   62.8703  -150.7625  111.30  2.1      ml   
4      2022-01-06T01:10:44.629Z   64.0145  -148.7657    1.90  1.5      ml   
...                         ...       ...        ...     ...  ...     ...   
71569  2022-06-04T15:26:13.439Z  -30.2315  -177.8443   39.37  4.6      mb   
71581  2022-06-04T14:02:17.078Z  -33.6929  -178.7322   10.00  4.4      mb   
71584  2022-06-04T13:51:07.024Z  -21.5817  -176.0672  144.23  4.6      mb   
71593  2022-06-04T13:04:18.850Z  -56.2831   -27.0051   65.64  4.8      mb   
73067  2022-05-30T10:46:28.887Z   22.2434   -70.3805   10.00  4.0      mb   

        nst     gap   dmin     rms  ...                   updated  \
0     

In [35]:
#drop duplicates in sqlite data
model_df2.drop_duplicates(subset=['id'], inplace=True)
print(model_df2)

                           time  latitude  longitude   depth  mag magType  \
0      2022-01-06T01:42:39.855Z   37.3840  -117.1075    1.10  1.3      ml   
1      2022-01-06T01:31:33.698Z   38.1810  -117.8180    7.30  1.4      ml   
2      2022-01-06T01:24:20.083Z   61.3407  -149.8852   30.00  1.5      ml   
3      2022-01-06T01:15:15.851Z   62.8703  -150.7625  111.30  2.1      ml   
4      2022-01-06T01:10:44.629Z   64.0145  -148.7657    1.90  1.5      ml   
...                         ...       ...        ...     ...  ...     ...   
71314  2022-06-04T15:26:13.439Z  -30.2315  -177.8443   39.37  4.6      mb   
71326  2022-06-04T14:02:17.078Z  -33.6929  -178.7322   10.00  4.4      mb   
71329  2022-06-04T13:51:07.024Z  -21.5817  -176.0672  144.23  4.6      mb   
71338  2022-06-04T13:04:18.850Z  -56.2831   -27.0051   65.64  4.8      mb   
72812  2022-05-30T10:46:28.887Z   22.2434   -70.3805   10.00  4.0      mb   

        nst     gap   dmin     rms  ...                   updated  \
0     

In [36]:
#replace the table in postgres with one without duplicates
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

583

In [37]:
#replace the table in sqlite with one without duplicates
model_df2.to_sql('earthquake_raw', conn, if_exists='replace', index = False)

64583

In [ ]:
#copy data as csv as backup method
model_df.to_csv("../Database/csv/earthquake_raw.csv",index=False)

In [ ]:
# Close connection to SQLite database
conn.close()
connection.close()